In [1]:
import pandas as pd

In [2]:
digits='0123456789'

In [3]:
def get_type_size(t):
    s=[i for i in t if i in digits]
    return int(''.join(s))

def get_simd_size(t):
    return int(t.split('x')[-1])

def get_simd_type(t):
    return t.split('x')[0]

def get_base_type(t):
    s=[i for i in t if i not in digits+'x']
    return (''.join(s))



In [4]:
def identify_retval_table(df):
    return any('Vec' in str(c) for c in df.columns) and 'Ret' in df.columns

In [5]:
def identify_parameters_table(df):
    t=df.iloc[0, 0]
    if type(t)==str:
        return t.startswith('a')
    return False

In [6]:
def is_binop(t):
    return len([i for i in t.split(',') if i.strip()])>1

In [7]:
def get_retvals_tables(url, ops, default):
    ret={}
    for op in ops:
        tables=pd.read_html(url+op+'.html')
        retval_tables=[t for t in tables if identify_retval_table(t)]
        params=[t for t in tables if identify_parameters_table(t)]
        binop=True
        if params:
            binop=is_binop(params[-1].iloc[0, 0])
        retval_table=retval_tables[-1] if retval_tables else default(binop)
        ret[op]=retval_table
    return ret

In [8]:
types=pd.DataFrame([("uint", "uint8", 16),
("int", "int8", 16),
("uint", "uint16", 8),
("int", "int16", 8),
("uint", "uint32", 4),
("int", "int32", 4),
("uint", "uint64", 2),
("int", "int64", 2),
("float", "float32", 4),
("float", "float64", 2),
("uint", "uint8", 32),
("int", "int8", 32),
("uint", "uint16", 16),
("int", "int16", 16),
("uint", "uint32", 8),
("int", "int32", 8),
("uint", "uint64", 4),
("int", "int64", 4),
("float", "float32", 8),
("float", "float64", 4),
("uint", "uint8", 64),
("int", "int8", 64),
("uint", "uint16", 32),
("int", "int16", 32),
("uint", "uint32", 16),
("int", "int32", 16),
("uint", "uint64", 8),
("int", "int64", 8),
("float", "float32", 16),
("float", "float64", 8),], columns=['base_type', 'size_type', 'size'])

In [9]:
types['width']=types.size_type.apply(get_type_size)

In [10]:
mask_types=types.iloc[:]

In [11]:
mask_types['base_type']='mask_'+mask_types['base_type']
mask_types['size_type']='mask_'+mask_types['size_type']

In [12]:
types=pd.concat([types, mask_types]).reset_index(drop=True)

In [28]:
types['floating']=types.base_type.str.contains('float')
types['mask']=types.base_type.str.contains('mask')

types['simd_type']=types.size_type+'x'+types['size'].astype(str)

In [29]:
def default_retval(ints=True, floating=True, mask=False, binop=True):
    temp=types[(types.floating==floating) | (types.floating!=ints) | (types['mask']==mask)]
    simd_types=temp.simd_type.drop_duplicates().values
    if binop:
        return pd.DataFrame({'Ret':simd_types, 'Vec1':simd_types, 'Vec2':simd_types})
    return pd.DataFrame({'Ret':simd_types, 'Vec':simd_types})

In [30]:
#bitwise ops
bitops_base_url='https://p12tic.github.io/libsimdpp/v2.2-dev/libsimdpp/w/bitwise/'
bitops=[
    'bit_and',
'bit_andnot',
'bit_or',
'bit_xor',
'bit_not'
]

In [32]:
bitops_retvals=get_retvals_tables(bitops_base_url, bitops, default=lambda bo: default_retval(binop=bo))

In [33]:
#floating ops
floatops_base_url='https://p12tic.github.io/libsimdpp/v2.2-dev/libsimdpp/w/fp/'
floatops=['add',
 'sub',
 'mul',
 'div',
 'fmadd',
 'fmsub',
 'neg',
 'round',
 'trunc',
 'floor',
 'ceil',
 'cmp_eq',
 'cmp_neq',
 'cmp_lt',
 'cmp_gt',
 'cmp_le',
 'cmp_ge',
 'abs',
 'sign',
 'min',
 'max',
 'isnan',
 'isnan2',
 'sqrt',
 'rcp_e',
 'rcp_rh',
 'rsqrt_e',
 'rsqrt_rh',
 'reduce_add',
 'reduce_mul',
 'reduce_min',
 'reduce_max']

In [34]:
floatops_retvals=get_retvals_tables(floatops_base_url, floatops, default=lambda bo: default_retval(False, binop=bo))

In [35]:
#int ops
intops_base_url='https://p12tic.github.io/libsimdpp/v2.2-dev/libsimdpp/w/int/'
intops=['add',
 'sub',
 'add_sat',
 'sub_sat',
 'mul_lo',
 'mul_hi',
 'mull',
 'neg',
 'cmp_eq',
 'cmp_neq',
 'cmp_lt',
 'cmp_gt',
 'cmp_le',
 'cmp_ge',
 'abs',
 'avg',
 'avg_trunc',
 'min',
 'max',
 'shift_l',
 'shift_r',
 'reduce_add',
 'reduce_mul',
 'reduce_min',
 'reduce_max',
 'reduce_and',
 'reduce_or']

In [36]:
intops_retvals=get_retvals_tables(intops_base_url, intops, default=lambda bo: default_retval(True, False, binop=bo))

In [37]:
def get_type_combinations(ts):
    ts=[types[types.size_type==t] for t in ts]
    sizes=set(ts[0]['size'].values)
    for t in ts[1:]:
        sizes = sizes.intersection(set(t['size'].values))
    ts=[t[t['size'].isin(sizes)] for t in ts]
    for s in sizes:
        ret=[]
        for t in ts:
            ret.append(t[t['size']==s].iloc[0]['simd_type'])
        yield ret

In [38]:
def gen_bin_op(op, op_name, arg_type, ret_type, arg2_type=None):
    if arg2_type is None:
        return f'''proc {op}*(x, y: {arg_type}): {ret_type}  {{.header: simd, importcpp: "{op_name}(@)"}}'''

    return f'''proc {op}*(x: {arg_type}, y: {arg2_type}): {ret_type}  {{.header: simd, importcpp: "{op_name}(@)"}}'''

def gen_uni_op(op, op_name, arg_type, ret_type):
    return f'''proc {op}*(x: {arg_type}): {ret_type}  {{.header: simd, importcpp: "{op_name}(@)"}}'''


In [39]:
aliases={
    'add':'`+`',
    'sub':'`-`',
    'mul':'`*`',
    'div':'`/`',
    'mull':'`*`',
    'shift_l':'`<<`',
    'shift_r':'`>>`',
     'cmp_eq': '`==`',
     'cmp_neq': '`~=`',
     'cmp_lt': '`<`',
     'cmp_gt': '`>`',
     'cmp_le': '`<=`',
     'cmp_ge': '`>=`',
    'bit_and': '`&`',
    'bit_andnot': '`&~`',
    'bit_or': '`|`',
    'bit_xor': '`^`',
    'bit_not': '`~`',
}

In [40]:
keywords={'div':'divide'}

In [41]:
def name_op(op):
    if op in keywords:
        return keywords[op]
    return op

In [42]:
def gen_op(op, retvals, ):
    binop=retvals.shape[1]>2
    if binop:
        for i in range(len(retvals)):
            temp=retvals.iloc[i]
            for c in get_type_combinations([temp['Ret'], temp['Vec1'], temp['Vec2']]):
                output(gen_bin_op(name_op(op), op, c[1], c[0], c[2]))
                if op in aliases:
                    output(gen_bin_op(aliases[op], op, c[1], c[0], c[2]))
    else:
        for i in range(len(retvals)):
            temp=retvals.iloc[i].to_dict()
            vec=[v for k,v in temp.items() if 'Vec' in k][0]
            for c in get_type_combinations([temp['Ret'], vec]):
                output(gen_uni_op(name_op(op), op, c[1], c[0]))
                if op in aliases:
                    output(gen_uni_op(aliases[op], op, c[1], c[0]))

In [43]:
size_types=types[types['mask']==False].size_type.unique()

In [44]:
simd_types=types[types['mask']==False].simd_type.unique()

In [58]:
def gen_simd_type(base_type, size_type, size):
    simd_type=size_type+'x'+str(size)
    args=', '.join(f'x{i}' for i in range(size))
    output(f"""
##############################################
### BEGIN {base_type}, {size_type}, {size} ###
##############################################
    
type {simd_type}* {{.header: simd, importcpp: "{size_type}<{size}>".}} = object
type mask_{simd_type}* {{.header: simd, importcpp: "mask_{size_type}<{size}>".}} = object

proc Vector*({args}: {size_type}): {simd_type} {{.header: simd, importcpp: "make_{base_type}<{size_type}<{size}>>(@)".}}
proc splat_{simd_type}*(x: SIZE_TYPE): {simd_type} = Vector({', '.join([f'x.{size_type}' for _ in range(size)])})

proc to_mask*(x: {simd_type}): mask_{simd_type} {{.header: simd, importcpp: "@.to_mask()".}}

proc len*(x: {simd_type}): Natural = {size}
{'NEWLINE'.join([f'proc extract_{i}*(x: {simd_type}): {size_type}  {{.header: simd, importcpp: "extract<{i}, {size}>(@)"}}' for i in range(size)])}

proc `[]`*(x: {simd_type}, i: Natural): {size_type} =
  assert 0 <= i and i < {size} ###
  {''.join(["if i == 0:###NEWLINE    return extract_0(x)###NEWLINE"]+[f"NEWLINE  elif i == {i}:###NEWLINE    return extract_{i}(x)###" for i in range(1, size-1)]+[f"NEWLINE  else:###NEWLINE    return extract_{size-1}(x)###NEWLINE"])}
proc to_seq*(x: {simd_type}): seq[{size_type}] = @[{', '.join([f'x[{i}]' for i in range(size)])}]
proc to_array*(x: {simd_type}): auto = [{', '.join([f'x[{i}]' for i in range(size)])}]
proc `$`*(x: {simd_type}): string = "{simd_type}"& $(x.to_array)
proc to_{simd_type}*(x: seq[SIZE_TYPE]): {simd_type} =
  assert x.len == {size} ###
  Vector({', '.join([f'x[{i}].{size_type}' for i in range(size)])})

############################################
### END {base_type}, {size_type}, {size} ###
############################################

    """.replace('NEWLINE', '\n'))

In [59]:
def gen_conversions():
    ret=[]
    for t1 in simd_types:
        for t2 in simd_types:
            size1=get_simd_size(t1)
            size2=get_simd_size(t2)
            t=get_simd_type(t2)
            if size1==size2:
                if t1!=t2:
                    output(
                        f'proc to_{t}*(x: {t1}): {t2} {{.header: simd, importcpp: "@.to_{t}()".}}'
                    )

In [60]:
def gen_simd_types(s):
#     import pdb; pdb.set_trace()
    if not s['mask']:
        gen_simd_type(s['base_type'], s['size_type'], s['size'])

In [61]:
all_output=""
def output(s):
    global all_output
    all_output+=s+'\n'

In [62]:
_=types.apply(gen_simd_types, axis=1)

In [63]:
gen_conversions()

In [64]:
for retvals in [bitops_retvals, floatops_retvals, intops_retvals]:
    for k, v in retvals.items():
        gen_op(k, v)

In [65]:
def dedup(ls):
    curr=[]
    for l in ls:
        if l not in curr:
            curr.append(l)
        elif "###" in l:
            curr.append(l)
    return curr


with open('./src/nimd.nim', 'a+') as f:
    f.write('''
const libsimdpppath {.strdefine.}: string = ""
{.localPassc: libsimdpppath.}
const simd = "<simdpp/simd.h>"
{.emit: """
#include <simdpp/simd.h>
using namespace simdpp;
""".}

'''+f"type SIZE_TYPE = {' or '.join(size_types)}\n\n")
    for l in dedup([i for i in all_output.split('\n') if i.strip()]):
        f.write(l)
        f.write('\n')